In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#@markdown #**Проверка GPU**
#@markdown ### Factory reset runtime если у вас не желаемый GPU.

#@markdown ---




#@markdown V100 = Превосходно (*Доступно только пользователям Colab Pro*)

#@markdown P100 = Очень хорошо

#@markdown T4 =  Намальна (*сойдет*)

#@markdown K80 = Ну, такое

#@markdown P4 = (*Не рекомендую*) 

#@markdown ---

!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-98a14698-f034-d7ce-19e1-61fb4ac81c32)


In [ ]:
#@markdown #**Анти-Отключение в Google Colab**
#@markdown ## Запустите это чтобы предотвратить автоматичское отключение
#@markdown  * Free пользователи получают примерно 6 - 12 часов;
#@markdown  * Pro пользователи получают примерно 24 часов (зависит).
#@markdown
#@markdown ---

import IPython
js_code = '''
    function ClickConnect(){
        console.log("Working");
        document.querySelector("colab-toolbar-button#connect").click()
    }
    setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

# Подключаем библиотеки и модельки

In [8]:
# @title Подключаем VQGAN и Clip

working_dir = "/content/drive/Shareddrives/ML/AIIJC/Solution for Audiobook" #@param {type:"string"}

print("Скачиваем CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null

print("Скачиваем Python AI библиотеки...")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
 
print("Устанавливаем библиотеки для обработки мета данных...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null
 
print("Устанавливаем обработку видео в Python...")
!pip install imageio-ffmpeg &> /dev/null
path = f'{working_dir}/steps'
!mkdir --parents {path}
print("Загрузка завершена.")

Устанавливаем обработку видео в Python...
Загрузка завершена.


In [4]:
# @title Кастомные функции для VQGAN
 
import argparse
import math
from pathlib import Path
import sys
import os
import cv2
import pandas as pd
import numpy as np
import subprocess
 
sys.path.append('./taming-transformers')

# Some models include transformers, others need explicit pip install
try:
    import transformers
except Exception:
    !pip install transformers
    import transformers

from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
 
from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
from imgtag import ImgTag    # metadata 
from libxmp import *         # metadata
import libxmp                # metadata
from stegano import lsb
import json
ImageFile.LOAD_TRUNCATED_IMAGES = True
 
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]
 
 
def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
 
 
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1
 
 
    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model
 
 
def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

     |████████████████████████████████| 2.9 MB 9.0 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
     |████████████████████████████████| 3.3 MB 66.2 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 109 kB 80.8 MB/s 
     |████████████████████████████████| 546 kB 71.9 MB/s 


In [66]:
#@title Подключаем библиотеки ruGPT (30 сек)
#@markdown ### и модель
%%capture

!pip install happytransformer

from happytransformer import HappyGeneration
from happytransformer import GENSettings

import re

model_path = "sberbank-ai/rugpt3medium_based_on_gpt2" #@param {type:"string"}

happy_gen = HappyGeneration(load_path = model_path)

RuntimeError: ignored

In [11]:
#@title Подключаем библиотеки Silero

%%capture

!pip install pytorch
!pip install -q torchaudio omegaconf
!pip install spacy
!pip install spacy_syllables
!python -m spacy download ru_core_news_md

import torch

In [12]:
#@title Генеральные функции
import spacy
from spacy_syllables import SpacySyllables
nlp = spacy.load('ru_core_news_md')
syllables = SpacySyllables(nlp)
nlp.add_pipe('syllables')

from IPython.display import display, Audio
from scipy.io.wavfile import write

def displayAudio(audio, rate = 16000):
  display(Audio(audio, rate=rate))

def createAudioFile(audio, name, rate = 16000):
  write(f'{name[:60]}.wav', rate, audio)

def generateRawAudio(speakerLines, lines, language = 'ru'):
  wavs = []
  for i in range(len(lines)):
    speaker = speakerLines[i]
    line = lines[i]

    device = torch.device('cpu')
    model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                          model='silero_tts',
                                                                          language=language,
                                                                          speaker=f"{speaker}_16khz")
    model = model.to(device)  # gpu or cpu
    audio = apply_tts(texts=[line],
                      model=model,
                      sample_rate=sample_rate,
                      symbols=symbols,
                      device=device)

    wavs.append({"audio": audio[0].numpy()}) # audio[0] since their is only one line
  return wavs

def createFullAudio(rawAudioWavs, textLines, rate = 16000):
  full_audio = np.array([])

  pause_length = 1000
  pause = np.array([0] * pause_length)

  for i, obj in enumerate(rawAudioWavs):
      obj["time_length"] = (obj["audio"].shape[0] + pause_length) / rate # seconds
      obj["line"] = textLines[i]
      full_audio = np.concatenate((full_audio, pause, obj["audio"]))
  return full_audio

def spacy_syllablize(word):
    token = nlp(word)[0]
    return token._.syllables

def isLastWord(word):
  return word[-1] in [',', '.', '!', '?', ':', ';']

def findFirstVowel(syllable):
  cnt = 0
  for letter in syllable:
    if letter in 'ёуеыаоэяию':
      return cnt 
    cnt += 1
  return -1

def stressText(text):
  res = ""
  words = text.split(' ')

  cnt = 0
  for word in words:
      endSentence = isLastWord(word)

      # Break the word to syllables and mark words if needed
      syllables = spacy_syllablize(word)
      print(syllables)
      for syllable in syllables:
          pos = findFirstVowel(syllable)
          if pos != -1:
            sign = "+" if cnt % 3 == 1 else ""
            res += f"{syllable[:pos]}{sign}{syllable[pos:]}"
            cnt += 1
          else:
            res += syllable

      if endSentence:
        res += word[-1]
        cnt = 0
      res += " "
  return res


/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

# Всяческие настройки

In [13]:
#@title Выбор модели VQGAN

#@markdown ---

imagenet_1024 = True #@param {type:"boolean"}
imagenet_16384 = False #@param {type:"boolean"}
gumbel_8192 = False #@param {type:"boolean"}
coco = False #@param {type:"boolean"}
faceshq = False #@param {type:"boolean"}
wikiart_1024 = False #@param {type:"boolean"}
wikiart_16384 = False #@param {type:"boolean"}
sflckr = False #@param {type:"boolean"}
ade20k = False #@param {type:"boolean"}
ffhq = False #@param {type:"boolean"}
celebahq = False #@param {type:"boolean"}

if imagenet_1024:
  !curl -L -o vqgan_imagenet_f16_1024.yaml -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 1024
  !curl -L -o vqgan_imagenet_f16_1024.ckpt -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fckpts%2Flast.ckpt&dl=1'  #ImageNet 1024
if imagenet_16384:
  !curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 16384
  !curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #ImageNet 16384
if gumbel_8192:
  !curl -L -o gumbel_8192.yaml -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #Gumbel 8192
  !curl -L -o gumbel_8192.ckpt -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #Gumbel 8192
if coco:
  !curl -L -o coco.yaml -C - 'https://dl.nmkd.de/ai/clip/coco/coco.yaml' #COCO
  !curl -L -o coco.ckpt -C - 'https://dl.nmkd.de/ai/clip/coco/coco.ckpt' #COCO
if faceshq:
  !curl -L -o faceshq.yaml -C - 'https://drive.google.com/uc?export=download&id=1fHwGx_hnBtC8nsq7hesJvs-Klv-P0gzT' #FacesHQ
  !curl -L -o faceshq.ckpt -C - 'https://app.koofr.net/content/links/a04deec9-0c59-4673-8b37-3d696fe63a5d/files/get/last.ckpt?path=%2F2020-11-13T21-41-45_faceshq_transformer%2Fcheckpoints%2Flast.ckpt' #FacesHQ
if wikiart_1024: 
  !curl -L -o wikiart_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart.yaml' #WikiArt 1024
  !curl -L -o wikiart_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart.ckpt' #WikiArt 1024
if wikiart_16384: 
  !curl -L -o wikiart_16384.yaml -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.yaml' #WikiArt 16384
  !curl -L -o wikiart_16384.ckpt -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.ckpt' #WikiArt 16384
if sflckr:
  !curl -L -o sflckr.yaml -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fconfigs%2F2020-11-09T13-31-51-project.yaml&dl=1' #S-FLCKR
  !curl -L -o sflckr.ckpt -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fcheckpoints%2Flast.ckpt&dl=1' #S-FLCKR
if ade20k:
  !curl -L -o ade20k.yaml -C - 'https://static.miraheze.org/intercriaturaswiki/b/bf/Ade20k.txt' #ADE20K
  !curl -L -o ade20k.ckpt -C - 'https://app.koofr.net/content/links/0f65c2cd-7102-4550-a2bd-07fd383aac9e/files/get/last.ckpt?path=%2F2020-11-20T21-45-44_ade20k_transformer%2Fcheckpoints%2Flast.ckpt' #ADE20K
if ffhq:
  !curl -L -o ffhq.yaml -C - 'https://app.koofr.net/content/links/0fc005bf-3dca-4079-9d40-cdf38d42cd7a/files/get/2021-04-23T18-19-01-project.yaml?path=%2F2021-04-23T18-19-01_ffhq_transformer%2Fconfigs%2F2021-04-23T18-19-01-project.yaml&force' #FFHQ
  !curl -L -o ffhq.ckpt -C - 'https://app.koofr.net/content/links/0fc005bf-3dca-4079-9d40-cdf38d42cd7a/files/get/last.ckpt?path=%2F2021-04-23T18-19-01_ffhq_transformer%2Fcheckpoints%2Flast.ckpt&force' #FFHQ
if celebahq:
  !curl -L -o celebahq.yaml -C - 'https://app.koofr.net/content/links/6dddf083-40c8-470a-9360-a9dab2a94e96/files/get/2021-04-23T18-11-19-project.yaml?path=%2F2021-04-23T18-11-19_celebahq_transformer%2Fconfigs%2F2021-04-23T18-11-19-project.yaml&force' #CelebA-HQ
  !curl -L -o celebahq.ckpt -C - 'https://app.koofr.net/content/links/6dddf083-40c8-470a-9360-a9dab2a94e96/files/get/last.ckpt?path=%2F2021-04-23T18-11-19_celebahq_transformer%2Fcheckpoints%2Flast.ckpt&force' #CelebA-HQ

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   645  100   645    0     0    714      0 --:--:-- --:--:-- --:--:--   714
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  913M  100  913M    0     0  14.7M      0  0:01:02  0:01:02 --:--:-- 14.9M


In [59]:
#@title Настройки GPT-Neo { run: "auto" }
#@markdown Используя `min_length` и `max_length` можно задать длинну видео по формуле
#@markdown $$\text{min_length} \cdot 2 / 190 \approx \text{длинна видео в минутах}$$
#@markdown `max_length` можно взять на 50 больше чем `min_length`.

# https://happytransformer.com/text-generation/settings/

min_length =  100#@param {type:"number"}
max_length =  150#@param {type:"number"}
do_sample = True #@param {type:"boolean"}
early_stopping = False #@param {type:"boolean"}
num_beams = 1 #@param {type:"number"}
temperature = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
top_k = 50 #@param {type:"slider", min:1, max:50, step:1}
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
no_repeat_ngram_size = 2 #@param {type:"slider", min:0, max:10, step:1}

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [15]:
#@title Настройки Silero { run: "auto" }
#@markdown Выберите голоса для озвучки (можно несколько).

import IPython

speakers = []

aidar = True #@param {type:"boolean"}
baya = False #@param {type:"boolean"}
irina = False #@param {type:"boolean"}
kseniya = True #@param {type:"boolean"}
natasha = False #@param {type:"boolean"}

if aidar: speakers.append("aidar")
if baya: speakers.append("baya")
if irina: speakers.append("irina")
if kseniya: speakers.append("kseniya")
if natasha: speakers.append("natasha")

if len(speakers) == 0:
  display(IPython.display.HTML('''
  <img width="300"src="..." alt="Listen here you little sh*t" />
  <br>
  You have to choose atleast one speaker
  '''))



# Генерация Шоу

In [67]:
# @title Параметры текста

name =  "\u042F \u043F\u043E\u043C\u043D\u044E \u0447\u0443\u0434\u043D\u043E\u0435 \u043C\u0433\u043D\u043E\u0432\u0435\u043D\u044C\u0435" #@param{type:"string"}

name = name.lower()

prompt = f"""Стих от Пушкина {name} """

# prompt = name

In [68]:
generateText()

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


RuntimeError: ignored

In [64]:
# @title Генерация и форматирование текста
def generateText():
  return happy_gen.generate_text(prompt, args=gen_args).text

def formatText(text):
    newText = text.replace("<nl>", "\n").replace("\'", "'").replace(", ", ",\n")
    newText = re.sub("[^а-яА-Я,.\n\s]", "", newText)
    newText = newText.replace(".,", ".").replace(". \n", ".").replace(".\n", ".").replace(",\n", ".").replace(",", ".").replace("\n", ".")

    sentences = newText.split(".")

    print(sentences)

    lines = [line for line in sentences if len(line) > 2]

    return [stressText(line) for line in lines]

rawGeneratedText = generateText()
textLines = formatText(rawGeneratedText) # Array of sentences



<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<input>:7: DeprecationWarning: invalid escape sequence \s
<ipython-input-64-592ce5bfccf2>:7: DeprecationWarning: invalid escape sequence \s
  newText = re.sub("[^а-яА-Я,.\n\s]", "", newText)
/usr/local/lib/python3.7/dist-packages/transformers/pip

RuntimeError: ignored

In [20]:
textLines

['xhttpswww ',
 'reddit ',
 'comrwritingpromptscommentskpywpyouhavealwayshadamotherorwifeandthatdqvroq ']

In [ ]:
textLines = [
    "Я помню чудное мгновенье:",
    "Передо мной явилась ты,",
    "Как мимолетное виденье,",
    "Как гений чистой красоты.",
    "В томленьях грусти безнадежной,",
    "В тревогах шумной суеты,",
    "Звучал мне долго голос нежный,",
    "И снились милые черты.",
    "Шли годы. Бурь порыв мятежный",
    "Рассеял прежние мечты,",
    "И я забыл твой голос нежный,",
    "Твои небесные черты.",
    "В глуши, во мраке заточенья",
    "Тянулись тихо дни мои",
    "Без божества, без вдохновенья,",
    "Без слёз, без жизни, без любви.",
    "Душе настало пробужденье:",
    "И вот опять явилась ты,",
    "Как мимолетное виденье,",
    "Как гений чистой красоты.",
    "И сердце бьется в упоенье,",
    "И для него воскресли вновь",
    "И божество, и вдохновенье,",
    "И жизнь, и слёзы, и любовь.",
]

textLines = [stressText(line) for line in textLines]

print(textLines)

In [ ]:
# @title Генерация озвучки
import random
import numpy as np

def assignSpeakersToLines(texts):
  result = (speakers * len(texts))[:len(texts)]

#   for i in range(len(texts)):
#     result.append(random.choice(speakers))
  return result

speakerLines = assignSpeakersToLines(textLines)

rawAudioWavs = generateRawAudio(speakerLines, textLines)

full_audio = createFullAudio(rawAudioWavs, textLines)
displayAudio(full_audio)
createAudioFile(full_audio, "kek")


In [ ]:
displayAudio(full_audio)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#@title Параметры генирации видео
#@markdown Можно оставить как оно есть. Что означают настройки описано [тут](https://colab.research.google.com/github/justinjohn0306/VQGAN-CLIP/blob/main/VQGAN%2BCLIP_(Zooming)_(z%2Bquantize_method_with_addons).ipynb#scrollTo=4INwpxFKgvbo).

fps = 12 #@param {type:"number"}
video_prompt = "0:("
for line in output_wav:
    video_prompt += line["line"] + ":0| "
video_prompt = video_prompt[:-2]
video_prompt += ")"

dur = 0
for i, line in enumerate(output_wav):

    if i > 0:
        video_prompt += f",{dur}:({line['line']}:1| {output_wav[i-1]['line']}:0)"
    else:
        video_prompt += f",{dur}:({line['line']}:1)"

    dur += int(line["time_length"] * fps)


key_frames = True
text_prompts = video_prompt
width =  512#@param {type:"number"}
height =  512#@param {type:"number"}
model = "vqgan_imagenet_f16_1024" #@param ["vqgan_imagenet_f16_16384", "vqgan_imagenet_f16_1024", "wikiart_1024", "wikiart_16384", "coco", "faceshq", "sflckr"]
interval =  2
initial_image = ""#@param {type:"string"}
target_images = ""#@param {type:"string"}
seed = 42
max_frames = ((dur + 50) // 2) *2
angle = "616: (0), 654:(9.5),693: (0), 1027: (0), 1065:(9.5), 1104: (0), 1467: (0), 1504:(9.5), 1542: (0), "#@param {type:"string"}
zoom = "0:(1.02), 300:(1.05), 305:(1.10), 1560:(1.25)"#@param {type:"string"}
translation_x = "0: (0)"#@param {type:"string"}
translation_y = "0: (0)"#@param {type:"string"}
iterations_per_frame = "0: (50), 50:(40), 100:(10), 1570:(1)"#@param {type:"string"}
save_all_iterations = False

model_names={
    "vqgan_imagenet_f16_16384": 'ImageNet 16384',
    "vqgan_imagenet_f16_1024":"ImageNet 1024", 
    "wikiart_1024":"WikiArt 1024",
    "wikiart_16384":"WikiArt 16384",
    "coco":"COCO-Stuff",
    "faceshq":"FacesHQ",
    "sflckr":"S-FLCKR"
}
model_name = model_names[model]

if seed == -1:
    seed = None

def parse_key_frames(string, prompt_parser=None):
    import re
    pattern = r'((?P<frame>[0-9]+):[\s]*[\(](?P<param>[\S\s]*?)[\)])'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param
    return frames

def get_inbetweens(key_frames, integer=False):
    key_frame_series = pd.Series([np.nan for a in range(max_frames)])
    for i, value in key_frames.items():
        key_frame_series[i] = value
    key_frame_series = key_frame_series.astype(float)
    key_frame_series = key_frame_series.interpolate(limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series

def split_key_frame_text_prompts(frames):
    prompt_dict = dict()
    for i, parameters in frames.items():
        prompts = parameters.split('|')
        for prompt in prompts:
            string, value = prompt.split(':')
            string = string.strip()
            value = float(value.strip())
            if string in prompt_dict:
                prompt_dict[string][i] = value
            else:
                prompt_dict[string] = {i: value}
    prompt_series_dict = dict()
    for prompt, values in prompt_dict.items():
        value_string = (
            ', '.join([f'{value}: ({values[value]})' for value in values])
        )
        prompt_series = get_inbetweens(parse_key_frames(value_string))
        prompt_series_dict[prompt] = prompt_series
    prompt_list = []
    for i in range(max_frames):
        prompt_list.append(
            ' | '.join(
                [f'{prompt}: {prompt_series_dict[prompt][i]}'
                 for prompt in prompt_series_dict]
            )
        )
    return prompt_list

if key_frames:
    text_prompts_series = split_key_frame_text_prompts(
        parse_key_frames(text_prompts)
    )
    target_images_series = split_key_frame_text_prompts(
        parse_key_frames(target_images)
    )
    angle_series = get_inbetweens(parse_key_frames(angle))
    zoom_series = get_inbetweens(parse_key_frames(zoom))
    translation_x_series = get_inbetweens(parse_key_frames(translation_x))
    translation_y_series = get_inbetweens(parse_key_frames(translation_y))
    iterations_per_frame_series = get_inbetweens(
        parse_key_frames(iterations_per_frame), integer=True
    )
else:
    text_prompts = [phrase.strip() for phrase in text_prompts.split("|")]
    if text_prompts == ['']:
        text_prompts = []
    if target_images == "None" or not target_images:
        target_images = []
    else:
        target_images = target_images.split("|")
        target_images = [image.strip() for image in target_images]

    angle = float(angle)
    zoom = float(zoom)
    translation_x = float(translation_x)
    translation_y = float(translation_y)
    iterations_per_frame = int(iterations_per_frame)

args = argparse.Namespace(
    prompts=text_prompts,
    image_prompts=target_images,
    noise_prompt_seeds=[],
    noise_prompt_weights=[],
    size=[width, height],
    init_weight=0.,
    clip_model='ViT-B/32',
    vqgan_config=f'{model}.yaml',
    vqgan_checkpoint=f'{model}.ckpt',
    step_size=0.1,
    cutn=64,
    cut_pow=1.,
    display_freq=interval,
    seed=seed,
)

In [ ]:
#@markdown **This cell deletes any frames already in the steps directory. Make sure you have saved any frames you want to keep from previous runs**

path = f'{working_dir}/steps'
!rm -r {path}
!mkdir --parents {path}

In [ ]:
#@markdown **Delete/clear the generated video**

if key_frames:
    # key frame filename would be too long
    filename = "video.mp4"
else:
    filename = f"{'_'.join(text_prompts).replace(' ', '')}.mp4"
filepath = f'{working_dir}/{filename}'

## Видео


In [ ]:
#@title Генерации кадров видео

# Delete memory from previous runs
!nvidia-smi -caa
for var in ['device', 'model', 'perceptor', 'z']:
  try:
      del globals()[var]
  except:
      pass

try:
    import gc
    gc.collect()
except:
    pass

try:
    torch.cuda.empty_cache()
except:
    pass

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if not key_frames:
    if text_prompts:
        print('Using text prompts:', text_prompts)
    if target_images:
        print('Using image prompts:', target_images)
if args.seed is None:
    seed = torch.seed()
else:
    seed = args.seed
torch.manual_seed(seed)
print('Using seed:', seed)
 
model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)
 
cut_size = perceptor.visual.input_resolution
e_dim = model.quantize.e_dim
f = 2**(model.decoder.num_resolutions - 1)
make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)
n_toks = model.quantize.n_e
toksX, toksY = args.size[0] // f, args.size[1] // f
sideX, sideY = toksX * f, toksY * f
z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]
stop_on_next_loop = False  # Make sure GPU memory doesn't get corrupted from cancelling the run mid-way through, allow a full frame to complete

for i in range(max_frames):
    if stop_on_next_loop:
      break
    if key_frames:
        text_prompts = text_prompts_series[i]
        text_prompts = [phrase.strip() for phrase in text_prompts.split("|")]
        if text_prompts == ['']:
            text_prompts = []
        args.prompts = text_prompts

        target_images = target_images_series[i]

        if target_images == "None" or not target_images:
            target_images = []
        else:
            target_images = target_images.split("|")
            target_images = [image.strip() for image in target_images]
        args.image_prompts = target_images

        angle = angle_series[i]
        zoom = zoom_series[i]
        translation_x = translation_x_series[i]
        translation_y = translation_y_series[i]
        iterations_per_frame = iterations_per_frame_series[i]
        print(
            f'text_prompts: {text_prompts}'
            f'angle: {angle}',
            f'zoom: {zoom}',
            f'translation_x: {translation_x}',
            f'translation_y: {translation_y}',
            f'iterations_per_frame: {iterations_per_frame}'
        )
    try:
        if i == 0 and initial_image != "":
            img_0 = cv2.imread(initial_image)
            z, *_ = model.encode(TF.to_tensor(img_0).to(device).unsqueeze(0) * 2 - 1)
        elif i == 0 and not os.path.isfile(f'{working_dir}/steps/{i:04d}.png'):
            one_hot = F.one_hot(
                torch.randint(n_toks, [toksY * toksX], device=device), n_toks
            ).float()
            z = one_hot @ model.quantize.embedding.weight
            z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
        else:
            if save_all_iterations:
                img_0 = cv2.imread(
                    f'{working_dir}/steps/{i:04d}_{iterations_per_frame}.png')
            else:
                # Hack to prevent colour inversion on every frame
                img_temp = cv2.imread(f'{working_dir}/steps/{i:04d}.png')
                imageio.imwrite('inverted_temp.png', img_temp)
                img_0 = cv2.imread('inverted_temp.png')
            center = (1*img_0.shape[1]//2, 1*img_0.shape[0]//2)
            trans_mat = np.float32(
                [[1, 0, translation_x],
                [0, 1, translation_y]]
            )
            rot_mat = cv2.getRotationMatrix2D( center, angle, zoom )

            trans_mat = np.vstack([trans_mat, [0,0,1]])
            rot_mat = np.vstack([rot_mat, [0,0,1]])
            transformation_matrix = np.matmul(rot_mat, trans_mat)

            img_0 = cv2.warpPerspective(
                img_0,
                transformation_matrix,
                (img_0.shape[1], img_0.shape[0]),
                borderMode=cv2.BORDER_WRAP
            )
            z, *_ = model.encode(TF.to_tensor(img_0).to(device).unsqueeze(0) * 2 - 1)
        i += 1

        z_orig = z.clone()
        z.requires_grad_(True)
        opt = optim.Adam([z], lr=args.step_size)

        normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                        std=[0.26862954, 0.26130258, 0.27577711])

        pMs = []

        for prompt in args.prompts:
            txt, weight, stop = parse_prompt(prompt)
            embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
            pMs.append(Prompt(embed, weight, stop).to(device))

        for prompt in args.image_prompts:
            path, weight, stop = parse_prompt(prompt)
            img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
            batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
            embed = perceptor.encode_image(normalize(batch)).float()
            pMs.append(Prompt(embed, weight, stop).to(device))

        for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
            gen = torch.Generator().manual_seed(seed)
            embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
            pMs.append(Prompt(embed, weight).to(device))

        def synth(z):
            z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
            return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

        def add_xmp_data(filename):
            imagen = ImgTag(filename=filename)
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
            if args.prompts:
                imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
            else:
                imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', model_name, {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.close()

        def add_stegano_data(filename):
            data = {
                "title": " | ".join(args.prompts) if args.prompts else None,
                "notebook": "VQGAN+CLIP",
                "i": i,
                "model": model_name,
                "seed": str(seed),
            }
            lsb.hide(filename, json.dumps(data)).save(filename)

        @torch.no_grad()
        def checkin(i, losses):
            losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
            tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
            out = synth(z)
            TF.to_pil_image(out[0].cpu()).save('progress.png')
            add_stegano_data('progress.png')
            add_xmp_data('progress.png')
            display.display(display.Image('progress.png'))

        def save_output(i, img, suffix=None):
            filename = \
                f"{working_dir}/steps/{i:04}{'_' + suffix if suffix else ''}.png"
            imageio.imwrite(filename, np.array(img))
            add_stegano_data(filename)
            add_xmp_data(filename)

        def ascend_txt(i, save=True, suffix=None):
            out = synth(z)
            iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

            result = []

            if args.init_weight:
                result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

            for prompt in pMs:
                result.append(prompt(iii))
            img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
            img = np.transpose(img, (1, 2, 0))
            if save:
                save_output(i, img, suffix=suffix)
            return result

        def train(i, save=True, suffix=None):
            opt.zero_grad()
            lossAll = ascend_txt(i, save=save, suffix=suffix)
            if i % args.display_freq == 0 and save:
                checkin(i, lossAll)
            loss = sum(lossAll)
            loss.backward()
            opt.step()
            with torch.no_grad():
                z.copy_(z.maximum(z_min).minimum(z_max))

        with tqdm() as pbar:
            if iterations_per_frame == 0:
                save_output(i, img_0)
            j = 1
            while True:
                suffix = (str(j) if save_all_iterations else None)
                if j >= iterations_per_frame:
                    train(i, save=True, suffix=suffix)
                    break
                if save_all_iterations:
                    train(i, save=True, suffix=suffix)
                else:
                    train(i, save=False, suffix=suffix)
                j += 1
                pbar.update()
    except KeyboardInterrupt:
      stop_on_next_loop = True
      pass

### Увеличение качества кадров (не обязательно запускать)

In [ ]:
#@markdown **Скачиваем модель SRCNN** 

!git clone https://github.com/Mirwaisse/SRCNN.git
!curl https://raw.githubusercontent.com/justinjohn0306/SRCNN/master/models/model_2x.pth -o model_2x.pth
!curl https://raw.githubusercontent.com/justinjohn0306/SRCNN/master/models/model_3x.pth -o model_3x.pth
!curl https://raw.githubusercontent.com/justinjohn0306/SRCNN/master/models/model_4x.pth -o model_4x.pth

In [ ]:
# @title **Улучшаем качество**

# Set zoomed = True if this cell is run
zoomed = True

zoom_type = 2 #@param [2, 3, 4]

for i in range(init_frame, last_frame): #
    filename = f"{i:04}.png"
    cmd = [
        'python',
        '/content/SRCNN/run.py',
        '--zoom_factor',
        '2',  # Note if you increase this, you also need to change the model.
        '--model',
        f'/content/model_{zoom_type}x.pth',  # 2x, 3x and 4x are available (change this line to you needs)
        '--image',
        filename,
        '--cuda'
    ]
    print(f'Улучшаем кадр {i}')

    process = subprocess.Popen(cmd, cwd=f'{working_dir}/steps/')
    stdout, stderr = process.communicate()
    if process.returncode != 0:
      raise RuntimeError(stderr)
      break

### Объединение в видео

In [ ]:
# @title Собираем кадры в видео

init_frame = 2
last_frame = max_frames
# fps = 12#@param {type:"number"}

frames = []
# tqdm.write('Generating video...')
try:
    zoomed
except NameError:
    image_path = f'{working_dir}/steps/%04d.png'
else:
    image_path = f'{working_dir}/steps/zoomed_%04d.png'

cmd = [
    'ffmpeg',
    '-y',
    '-vcodec',
    'png',
    '-r',
    str(fps),
    '-start_number',
    str(init_frame),
    '-i',
    image_path,
    '-c:v',
    'libx264',
    '-vf',
    f'fps={fps}',
    '-pix_fmt',
    'yuv420p',
    '-crf',
    '17',
    '-preset',
    'veryslow',
    filepath
]

process = subprocess.Popen(cmd, cwd=f'{working_dir}/steps/', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    raise RuntimeError(stderr)
else:
    print("Видео готово")

### Интерполяция кадров (не обязательно запускать)
Воспользуемся Super-Slomo моделью как интерполятор кадров. Новый FPS будет равен исходному FPS умноженному на интерполяционный фактор.

In [ ]:
# @title **Download Super-Slomo model**
!git clone -q --depth 1 https://github.com/avinashpaliwal/Super-SloMo.git
from os.path import exists
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  
pretrained_model = 'SuperSloMo.ckpt'
if not exists(pretrained_model):
  download_from_google_drive('1IvobLDbRiBgZr3ryCRrWL8xDbMZ-KnpF', pretrained_model)

In [ ]:
INTERP_FACTOR = 3 #@param {type:"number"}
TARGET_FPS = fps * INTERP_FACTOR

cmd1 = [
    'python',
    'Super-SloMo/video_to_slomo.py',
    '--checkpoint',
    pretrained_model,
    '--video',
    filepath,
    '--sf',
    str(INTERP_FACTOR),
    '--fps',
    str(TARGET_FPS),
    '--output',
    f'{filepath}-slomo.mkv',
]
process = subprocess.Popen(cmd1, cwd=f'/content', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    raise RuntimeError(stderr)

cmd2 = [
    'ffmpeg',
    '-i',
    f'{filepath}-slomo.mkv',
    f'{filepath}-slomo.mp4',
]

process = subprocess.Popen(cmd2, cwd=f'/content', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    raise RuntimeError(stderr)


## Объединение Аудио и Видео

In [ ]:
pip install ffmpeg-python

In [ ]:
import ffmpeg

video = ffmpeg.input(f'{name}.mp4')
audio = ffmpeg.input(f'{name}.wav')
out = ffmpeg.output(video, audio, "combined.mp4", vcodec='copy', acodec='aac', strict='experimental')
out.run()

(None, None)